In [1]:
import pickle
import numpy as np
import pandas as pd
import os
from tqdm.notebook import tqdm
import re

seed = 2023

In [2]:
from nlpsig_networks.scripts.swnu_network_functions import (
    swnu_network_hyperparameter_search, obtain_SWNUNetwork_input
)

In [3]:
output_dir = "rumours_output"
if not os.path.isdir(output_dir):
    os.makedirs(output_dir)

## Rumours

In [4]:
%run load_sbert-embeddings.py

In [5]:
df_rumours.head()

,id,label,datetime,text,timeline_id,set
0,5.249902e+17,0,2014-10-22 18:26:23,Police have clarified that there were two shoo...,0,train
1,5.249906e+17,0,2014-10-22 18:27:58,"@CTVNews you guys ""confirmed"" there were 3 sho...",0,train
2,5.249908e+17,1,2014-10-22 18:28:46,@CTVNews get it right. http://t.co/GHYxMuzPG9,0,train
3,5.249927e+17,1,2014-10-22 18:36:29,RT @CTVNews Police have clarified that there w...,0,train
4,5.250038e+17,1,2014-10-22 19:20:41,@CTVNews @ctvsaskatoon so what happened at Rid...,0,train


In [6]:
x_data = obtain_SWNUNetwork_input(
    method = "gaussian_random_projection",
    dimension= 30,
    df= df_rumours,
    id_column='timeline_id',
    label_column='label',
    embeddings= sbert_embeddings,
    k=5,
    features='time_encoding',
    standardise_method=None,
    add_time_in_path=False,
)

x_data[0].shape

[INFO] Concatenating the embeddings to the dataframe...
[INFO] - columns beginning with 'e' denote the full embddings.
[INFO] - columns beginning with 'd' denote the dimension reduced embeddings.
[INFO] Adding time feature columns into dataframe in `.df`.
[INFO] Adding 'time_encoding' feature...
[INFO] Adding 'time_diff' feature...
[INFO] Adding 'timeline_index' feature...
[INFO] Padding ids and storing in `.df_padded` and `.array_padded` attributes.


  0%|          | 0/5568 [00:00<?, ?it/s]

[INFO] The path was created for each item in the dataframe, by looking at its history, so to include embeddings in the FFN input, we concatenate the embeddings for each sentence / text.


torch.Size([5568, 5, 415])

## SWNU Network

In [7]:
features = ["time_encoding", "timeline_index"]
standardise_method = ["z_score", None]
num_features = len(features)
add_time_in_path = True

In [8]:
num_epochs = 100
embedding_dim = 384
dimensions = [15]
swnu_hidden_dim_sizes_and_sig_depths = [([12], 3), ([10], 4)]
ffn_hidden_dim_sizes = [[64,64],[128,128],[256,256],[512,512]]
dropout_rates = [0.5, 0.1]
learning_rates = [1e-3, 1e-4, 5e-4]
seeds = [1, 12, 123]
bidirectional = True
loss = "focal"
gamma = 2
validation_metric = "f1"
patience = 5
split_indices = (df_rumours[df_rumours['set']=='train'].index,
                 df_rumours[df_rumours['set']=='dev'].index,
                 df_rumours[df_rumours['set']=='test'].index)

In [9]:
# #dimensionality reduction
# embedding_dim = sbert_embeddings.shape[1]
# dim_reduce_method = ["umap"] #["gaussian_random_projection", "umap"]
# dimensions = [20]#[50,30,15]
# #time features
# features = "time_encoding"
# standardise_method = "standardise"
# add_time_in_path = False
# #SWNU block
# augmentation_tp = "Conv1d"
# hidden_dim_aug = None
# comb_m = "concatenation"
# log_sig = True
# conv_output_channels = [10] #[20, 10, 5]
# log_signature_dimensions_and_sig_depths = [(8, 3)]#[(30, 2), (10, 3), (6, 4)]
# bidirectional = False
# #ffn
# hidden_dim_sizes = [64]#[32,64]
# dropout_rates = [0.2]#[0.5, 0.2, 0.1]
# #overall training
# num_epochs = 100
# batch=64
# patience = 4
# learning_rates = [1e-4] #[1e-3, 1e-4, 5e-4]
# seeds = [0, 1, 12, 123, 1234]
# loss = "focal"
# gamma = 2
# validation_metric = "f1"
# split_indices = (df_rumours[df_rumours['set']=='train'].index,
#                  df_rumours[df_rumours['set']=='dev'].index,
#                  df_rumours[df_rumours['set']=='test'].index)

In [10]:
size=35

## UMAP

In [11]:
swnu_network_umap, best_swnu_network_umap, _, __ = swnu_network_hyperparameter_search(
    num_epochs=num_epochs,
    df=df_rumours,
    id_column="timeline_id",
    label_column="label",
    embeddings=sbert_embeddings,
    y_data=y_data,
    embedding_dim=embedding_dim,
    output_dim=output_dim,
    history_lengths=[size],
    dim_reduce_methods=["umap"],
    dimensions=dimensions,
    log_signature=True,
    swnu_hidden_dim_sizes_and_sig_depths=swnu_hidden_dim_sizes_and_sig_depths,
    ffn_hidden_dim_sizes=ffn_hidden_dim_sizes,
    dropout_rates=dropout_rates,
    learning_rates=learning_rates,
    BiLSTM=bidirectional,
    seeds=seeds,
    loss=loss,
    gamma=gamma,
    features=features, 
    standardise_method=standardise_method,
    add_time_in_path=add_time_in_path,
    split_ids=None, #torch.tensor(df_rumours['timeline_id'].astype(int)),
    split_indices=split_indices,
    k_fold=False,
    patience=patience,
    validation_metric=validation_metric,
    results_output=f"{output_dir}/swnu_network_umap_focal_{gamma}_{size}.csv",
    verbose=False
)

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]


##################################################
dimension: 15 | method: umap
[INFO] Concatenating the embeddings to the dataframe...
[INFO] - columns beginning with 'e' denote the full embddings.
[INFO] - columns beginning with 'd' denote the dimension reduced embeddings.
[INFO] Adding time feature columns into dataframe in `.df`.
[INFO] Adding 'time_encoding' feature...
[INFO] Adding 'time_diff' feature...
[INFO] Adding 'timeline_index' feature...
[INFO] Padding ids and storing in `.df_padded` and `.array_padded` attributes.


  0%|          | 0/5568 [00:00<?, ?it/s]

...

  0%|          | 0/3 [00:00<?, ?it/s]

  0%|          | 0/3 [00:00<?, ?it/s]

[INFO] Concatenating the embeddings to the dataframe...
[INFO] - columns beginning with 'e' denote the full embddings.
[INFO] - columns beginning with 'd' denote the dimension reduced embeddings.
[INFO] Adding time feature columns into dataframe in `.df`.
[INFO] Adding 'time_encoding' feature...
[INFO] Adding 'time_diff' feature...
[INFO] Adding 'timeline_index' feature...
[INFO] Padding ids and storing in `.df_padded` and `.array_padded` attributes.


  0%|          | 0/5568 [00:00<?, ?it/s]

[INFO] The path was created for each item in the dataframe, by looking at its history, so to include embeddings in the FFN input, we concatenate the embeddings for each sentence / text.
saving results dataframe to CSV for this hyperparameter search in rumours_output/swnu_network_umap_focal_2_35.csv
saving the best model results dataframe to CSV for this hyperparameter search in rumours_output/swnu_network_umap_focal_2_35_best_model.csv


In [12]:
swnu_network_umap.groupby(["dimensions",
                           "swnu_hidden_dim",
                           "ffn_hidden_dim",
                           "dropout_rate",
                           "learning_rate"]).mean()

/tmp/ipykernel_1937203/2289302539.py:1: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  swnu_network_umap.groupby(["dimensions",


loss  \
dimensions swnu_hidden_dim ffn_hidden_dim dropout_rate learning_rate             
15         (10,)           (64, 64)       0.1          0.0001         0.371781   
                                                       0.0005         0.440356   
                                                       0.0010         0.360259   
                                          0.5          0.0001         0.377169   
                                                       0.0005         0.455574   
                                                       0.0010         0.348458   
                           (128, 128)     0.1          0.0001         0.524425   
                                                       0.0005         0.412525   
                                                       0.0010         0.473316   
                                          0.5          0.0001         0.350557   
                                                       0.0005         0.427026   
                                                       0.0010         0.327612   
                           (256, 256)     0.1          0.0001         0.455113   
                                                       0.0005         0.383629   
                                                       0.0010         0.402671   
                                          0.5          0.0001         0.414512   
                                                       0.0005         0.390813   
                                                       0.0010         0.432883   
                           (512, 512)     0.1          0.0001         0.559197   
                                                       0.0005         0.489585   
                                                       0.0010         0.482960   
                                          0.5          0.0001         0.438350   
                                                       0.0005         0.347589   
                                                       0.0010         0.419916   
           (12,)           (64, 64)       0.1          0.0001         0.285349   
                                                       0.0005         0.446762   
                                                       0.0010         0.343697   
                                          0.5          0.0001         0.273866   
                                                       0.0005         0.426052   
                                                       0.0010         0.480820   
                           (128, 128)     0.1          0.0001         0.319503   
                                                       0.0005         0.450899   
                                                       0.0010         0.539012   
                                          0.5          0.0001         0.307972   
                                                       0.0005         0.440530   
                                                       0.0010         0.367332   
                           (256, 256)     0.1          0.0001         0.466642   
                                                       0.0005         0.569630   
                                                       0.0010         0.601132   
                                          0.5          0.0001         0.481612   
                                                       0.0005         0.461614   
                                                       0.0010         0.497822   
                           (512, 512)     0.1          0.0001         0.385423   
                                                       0.0005         0.541610   
                                                       0.0010         0.509491   
                                          0.5          0.0001         0.379065   
                                                       0.0005         0.504722   
                                                       0

In [13]:
best_swnu_network_umap

,loss,accuracy,f1,f1_scores,precision,precision_scores,recall,recall_scores,valid_loss,valid_accuracy,...,seed,BiLSTM,loss_function,gamma,k_fold,n_splits,augmentation_type,hidden_dim_aug,comb_method,batch_size
0,0.389666,0.609152,0.605639,"[0.5684210526315788, 0.6428571428571428]",0.621381,"[0.6733167082294265, 0.5694444444444444]",0.614902,"[0.4918032786885246, 0.738]",0.210545,0.715302,...,1,True,focal,2,False,None,Conv1d,None,concatenation,64
0,0.549847,0.628217,0.623035,"[0.5788336933045356, 0.6672354948805461]",0.646360,"[0.7108753315649867, 0.5818452380952381]",0.635080,"[0.48816029143898, 0.782]",0.391679,0.676157,...,12,True,focal,2,False,None,Conv1d,None,concatenation,64
0,0.381554,0.643470,0.642096,"[0.6199186991869919, 0.6642728904847397]",0.651878,"[0.7011494252873564, 0.6026058631921825]",0.647778,"[0.5555555555555556, 0.74]",0.449311,0.729537,...,123,True,focal,2,False,None,Conv1d,None,concatenation,64


In [14]:
best_swnu_network_umap["f1"].mean()

0.6235898288909225

In [15]:
best_swnu_network_umap["precision"].mean()

0.6398728351356057

In [16]:
best_swnu_network_umap["recall"].mean()

0.6325865209471767

In [17]:
np.stack(best_swnu_network_umap["f1_scores"]).mean(axis=0)

array([0.58905782, 0.65812184])

In [18]:
np.stack(best_swnu_network_umap["precision_scores"]).mean(axis=0)

array([0.69511382, 0.58463185])

In [19]:
np.stack(best_swnu_network_umap["recall_scores"]).mean(axis=0)

array([0.51183971, 0.75333333])

## Unidirectional LSTM

In [ ]:
swnu_network_umap_uni, best_swnu_network_umap_uni, _, __ = swnu_network_hyperparameter_search(
    num_epochs=num_epochs,
    df=df_rumours,
    id_column="timeline_id",
    label_column="label",
    embeddings=sbert_embeddings,
    y_data=y_data,
    embedding_dim=embedding_dim,
    output_dim=output_dim,
    history_lengths=[size],
    dim_reduce_methods=["umap"],
    dimensions=dimensions,
    log_signature=True,
    swnu_hidden_dim_sizes_and_sig_depths=swnu_hidden_dim_sizes_and_sig_depths,
    ffn_hidden_dim_sizes=ffn_hidden_dim_sizes,
    dropout_rates=dropout_rates,
    learning_rates=learning_rates,
    BiLSTM=False,
    seeds=seeds,
    loss=loss,
    gamma=gamma,
    features=features, 
    standardise_method=standardise_method,
    add_time_in_path=add_time_in_path,
    split_ids=None, #torch.tensor(df_rumours['timeline_id'].astype(int)),
    split_indices=split_indices,
    k_fold=False,
    patience=patience,
    validation_metric=validation_metric,
    results_output=f"{output_dir}/swnu_network_umap_focal_{gamma}_{size}_uni.csv",
    verbose=False
)

In [ ]:
swnu_network_umap_uni.groupby(["dimensions",
                               "swnu_hidden_dim",
                               "ffn_hidden_dim",
                               "dropout_rate",
                               "learning_rate"]).mean()

In [ ]:
best_swnu_network_umap_uni

In [ ]:
best_swnu_network_umap_uni["f1"].mean()

In [ ]:
best_swnu_network_umap_uni["precision"].mean()

In [ ]:
best_swnu_network_umap_uni["recall"].mean()

In [ ]:
np.stack(best_swnu_network_umap_uni["f1_scores"]).mean(axis=0)

In [ ]:
np.stack(best_swnu_network_umap_uni["precision_scores"]).mean(axis=0)

In [ ]:
np.stack(best_swnu_network_umap_uni["recall_scores"]).mean(axis=0)

## GRP

In [ ]:
swnu_network_grp, best_swnu_network_grp, _, __ = swnu_network_hyperparameter_search(
    num_epochs=num_epochs,
    df=df_rumours,
    id_column="timeline_id",
    label_column="label",
    embeddings=sbert_embeddings,
    y_data=y_data,
    embedding_dim=embedding_dim,
    output_dim=output_dim,
    history_lengths=[size],
    dim_reduce_methods=["gaussian_random_projection"],
    dimensions=dimensions,
    log_signature=True,
    swnu_hidden_dim_sizes_and_sig_depths=swnu_hidden_dim_sizes_and_sig_depths,
    ffn_hidden_dim_sizes=ffn_hidden_dim_sizes,
    dropout_rates=dropout_rates,
    learning_rates=learning_rates,
    BiLSTM=bidirectional,
    seeds=seeds,
    loss=loss,
    gamma=gamma,
    features=features, 
    standardise_method=standardise_method,
    add_time_in_path=add_time_in_path,
    split_ids=None, #torch.tensor(df_rumours['timeline_id'].astype(int)),
    split_indices=split_indices,
    k_fold=False,
    patience=patience,
    validation_metric=validation_metric,
    results_output=f"{output_dir}/swnu_network_grp_focal_{gamma}_{size}.csv",
    verbose=False
)

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]


##################################################
dimension: 15 | method: gaussian_random_projection
[INFO] Concatenating the embeddings to the dataframe...
[INFO] - columns beginning with 'e' denote the full embddings.
[INFO] - columns beginning with 'd' denote the dimension reduced embeddings.
[INFO] Adding time feature columns into dataframe in `.df`.
[INFO] Adding 'time_encoding' feature...
[INFO] Adding 'time_diff' feature...
[INFO] Adding 'timeline_index' feature...
[INFO] Padding ids and storing in `.df_padded` and `.array_padded` attributes.


  0%|          | 0/5568 [00:00<?, ?it/s]

...

  0%|          | 0/3 [00:00<?, ?it/s]

  0%|          | 0/3 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/2 [00:00<?, ?it/s]

  0%|          | 0/3 [00:00<?, ?it/s]

In [ ]:
swnu_network_grp.groupby(["dimensions",
                           "swnu_hidden_dim",
                           "ffn_hidden_dim",
                           "dropout_rate",
                           "learning_rate"]).mean()

In [ ]:
best_swnu_network_grp

In [ ]:
best_swnu_network_grp["f1"].mean()

In [ ]:
best_swnu_network_grp["precision"].mean()

In [ ]:
best_swnu_network_grp["recall"].mean()

In [ ]:
np.stack(best_swnu_network_grp["f1_scores"]).mean(axis=0)

In [ ]:
np.stack(best_swnu_network_grp["precision_scores"]).mean(axis=0)

In [ ]:
np.stack(best_swnu_network_grp["recall_scores"]).mean(axis=0)

## Unidirectional LSTM

In [ ]:
swnu_network_grp_uni, best_swnu_network_grp_uni, _, __ = swnu_network_hyperparameter_search(
    num_epochs=num_epochs,
    df=df_rumours,
    id_column="timeline_id",
    label_column="label",
    embeddings=sbert_embeddings,
    y_data=y_data,
    embedding_dim=embedding_dim,
    output_dim=output_dim,
    history_lengths=[size],
    dim_reduce_methods=["gaussian_random_projection"],
    dimensions=dimensions,
    log_signature=True,
    swnu_hidden_dim_sizes_and_sig_depths=swnu_hidden_dim_sizes_and_sig_depths,
    ffn_hidden_dim_sizes=ffn_hidden_dim_sizes,
    dropout_rates=dropout_rates,
    learning_rates=learning_rates,
    BiLSTM=False,
    seeds=seeds,
    loss=loss,
    gamma=gamma,
    features=features, 
    standardise_method=standardise_method,
    add_time_in_path=add_time_in_path,
    split_ids=None, #torch.tensor(df_rumours['timeline_id'].astype(int)),
    split_indices=split_indices,
    k_fold=False,
    patience=patience,
    validation_metric=validation_metric,
    results_output=f"{output_dir}/swnu_network_grp_focal_{gamma}_{size}_uni.csv",
    verbose=False
)

In [ ]:
swnu_network_grp_uni.groupby(["dimensions",
                              "swnu_hidden_dim",
                              "ffn_hidden_dim",
                              "dropout_rate",
                              "learning_rate"]).mean()

In [ ]:
best_swnu_network_grp_uni

In [ ]:
best_swnu_network_grp_uni["f1"].mean()

In [ ]:
best_swnu_network_grp_uni["precision"].mean()

In [ ]:
best_swnu_network_grp_uni["recall"].mean()

In [ ]:
np.stack(best_swnu_network_grp_uni["f1_scores"]).mean(axis=0)

In [ ]:
np.stack(best_swnu_network_grp_uni["precision_scores"]).mean(axis=0)

In [ ]:
np.stack(best_swnu_network_grp_uni["recall_scores"]).mean(axis=0)